In [1]:
from google.cloud import bigquery

client = bigquery.Client(project="valued-lambda-478823-h0")

query = """
SELECT
*
FROM `valued-lambda-478823-h0.azdd_test.party_status_clean`
"""


df = client.query(query).to_dataframe()

df.head()

,county,active_acn,active_apv,active_dem,active_grn,active_lbr,active_rep,active_uaf,active_uni,active_total,...,prereg_acn,prereg_apv,prereg_dem,prereg_grn,prereg_lbr,prereg_rep,prereg_uaf,prereg_uni,prereg_total,grand_total
0,Cheyenne,4,0,127,0,1,801,291,0,1224,...,1,0,0,0,0,10,5,0,16,1393
1,Hinsdale,0,0,90,0,8,386,188,0,672,...,0,0,0,0,0,4,0,0,4,745
2,Kiowa,2,0,106,0,7,582,249,1,947,...,0,0,0,0,0,7,7,0,14,1022
3,Mineral,1,0,259,0,1,282,188,0,731,...,0,0,0,0,0,0,2,0,2,837
4,Sedgwick,8,0,257,1,4,871,407,0,1548,...,0,0,3,0,0,4,8,0,15,1746


In [2]:
import pandas as pd

id_cols = ["county"]

value_cols = [
    c for c in df.columns
    if c not in id_cols
    and not c.endswith("_total")
    and c != "grand_total"
]

long_df = df.melt(
    id_vars=id_cols,
    value_vars=value_cols,
    var_name="status_party",
    value_name="count"
)

long_df[["status", "party"]] = long_df["status_party"].str.split("_", n=1, expand=True)

long_df["party"] = long_df["party"].apply(
    lambda p: p if p in ["dem", "rep"] else "third"
)

long_df = long_df[["county", "status", "party", "count"]]

long_df.head()


,county,status,party,count
0,Cheyenne,active,third,4
1,Hinsdale,active,third,0
2,Kiowa,active,third,2
3,Mineral,active,third,1
4,Sedgwick,active,third,8


In [3]:
from google.cloud import bigquery

project = "valued-lambda-478823-h0"
dataset = "azdd_test"
table = f"{project}.{dataset}.party_status_long"

client = bigquery.Client(project=project)

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE"
)

load_job = client.load_table_from_dataframe(
    long_df,
    destination=table,
    job_config=job_config
)

load_job.result()

print(f"Loaded {client.get_table(table).num_rows} rows into {table}")

Loaded 1560 rows into valued-lambda-478823-h0.azdd_test.party_status_long
